In [4]:
import rtree.index as rti

idx = rti.Index()
idx.insert(0, (0,0,0,0))

In [13]:
import random
import math

def get_random_bounds(size_fraction=0.1):
    assert(size_fraction < 1)
    
    width_fraction = math.sqrt(size_fraction)
    
    x_pos= random.random() * (1 - width_fraction)
    y_pos = random.random() * (1 - width_fraction)
    
    
    x_width = random.random() * width_fraction
    y_width = random.random() * width_fraction
    
    #print('x_width:', x_width, "y_width:", y_width, x_width * y_width)
    return np.array([x_pos, y_pos, x_pos + x_width, y_pos + y_width])

In [14]:
import random
%timeit get_random_bounds()

2.04 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [28]:
def run_query_rtree(idx, size_fraction=0.1):
    #print(query_bounds)
    query_bounds = get_random_bounds(size_fraction)

    t1 = time.time()
    points = list(idx.intersection(query_bounds))
    #print("len(points):", len(points))
    t2 = time.time()

In [34]:
import numba as nb
    
@nb.jit
def add_to_rtree(idx, points):
    for i in range(len(points)):
        idx.insert(i, [points[i][0], points[i][1], points[i][0], points[i][1]])

In [44]:
import numpy as np
import time
import timeit
times = []

width = 1

"""for i in ([1000, 2000, 4000, 8000, 16000, 32000, 64000, 128000, 
           256000, 512000, 1024000, 2048000, 4096000,
          8192000, 16384000, 32768000, 65536000, 131072000]):"""
for i in range(5, 11):
    idx = rti.Index()

    num_points = 1000 * 2**i #100000000
    new_points = width * np.random.random((num_points,2))
    
    t1 = time.time()
    '''
    for i,point in enumerate(new_points):
        idx.insert(i, (point[0], point[1], point[0], point[1]))
    '''
    add_to_rtree(idx, new_points)
    t2 = time.time()
    #print("time1:", t2 - t1)
    
    num = 10
    ti_0001 = timeit.timeit(lambda: run_query_rtree(idx, 0.0001), number=num)
    ti_01 = timeit.timeit(lambda: run_query_rtree(idx, 0.01), number=num)
    ti_1 = timeit.timeit(lambda: run_query_rtree(idx, 0.1), number=num)

    times += [[num_points, t2-t1, 1000 * ti_0001 / num, 
               1000 * ti_01 / num, 1000 * ti_1 / num]]
    print(times[-1])
    
    

[32000, 2.3054189682006836, 0.05858599906787276, 0.1433265977539122, 0.8689333975780755]
[64000, 4.711745977401733, 0.07683219737373292, 0.23817699984647334, 1.0221403033938259]
[128000, 9.570508003234863, 0.07558770012110472, 0.4080764017999172, 3.825860598590225]
[256000, 20.193315982818604, 0.07677520043216646, 0.4927272966597229, 4.956748598488048]
[512000, 48.70453882217407, 0.09282010141760111, 1.2432204966899008, 8.007755997823551]
[1024000, 107.54522681236267, 0.20767340320162475, 2.6984305994119495, 30.070026696193963]
